In [1]:
!pip install torch torchvision transformers bert-score evaluate tqdm matplotlib numpy sklearn evaluate rouge_score

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
!unzip models.zip

Archive:  models.zip
   creating: models/
  inflating: models/wm_stegastamp_decoder.pth  
  inflating: models/wm_stegastamp_encoder.pth  
  inflating: models/stegastamp_wm.py  


In [3]:
!unzip data.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/images/train/000000157046.jpg  
  inflating: __MACOSX/data/images/train/._000000157046.jpg  
  inflating: data/images/train/000000421455.jpg  
  inflating: __MACOSX/data/images/train/._000000421455.jpg  
  inflating: data/images/train/000000525155.jpg  
  inflating: __MACOSX/data/images/train/._000000525155.jpg  
  inflating: data/images/train/000000413395.jpg  
  inflating: __MACOSX/data/images/train/._000000413395.jpg  
  inflating: data/images/train/000000579158.jpg  
  inflating: __MACOSX/data/images/train/._000000579158.jpg  
  inflating: data/images/train/000000276024.jpg  
  inflating: __MACOSX/data/images/train/._000000276024.jpg  
  inflating: data/images/train/000000077396.jpg  
  inflating: __MACOSX/data/images/train/._000000077396.jpg  
  inflating: data/images/train/000000436883.jpg  
  inflating: __MACOSX/data/images/train/._000000436883.jpg  
  inflating: data/images/train/000000419653.jpg  
  inflating

In [18]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
from tqdm import trange, tqdm
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score, recall_score, f1_score
from models.stegastamp_wm import StegaStampDecoder, StegaStampEncoder
import collections
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor,AutoTokenizer
from torch.utils.data import DataLoader


In [6]:
DATASET_SIZE = 1000
IMAGE_SIZE = 256
NUM_BITS = 48
IMAGE_CHANNELS = 3

if torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'


coco_dataset = datasets.CocoCaptions(root = './data/images/train',
                        annFile = './data/annotations/train_captions.json',
                        transform=transforms.Compose([
                            transforms.Resize(IMAGE_SIZE),
                            transforms.CenterCrop(IMAGE_SIZE),
                            transforms.ToTensor()
                        ]))


loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


In [7]:
signature = torch.randint(0, 2, (1, NUM_BITS), device=device).float()
wm_encoder = StegaStampEncoder(
    IMAGE_SIZE,
    IMAGE_CHANNELS,
    NUM_BITS,
)
wm_encoder_load = torch.load('models/wm_stegastamp_encoder.pth', map_location=device, weights_only=True)
if type(wm_encoder_load) is collections.OrderedDict:
    wm_encoder.load_state_dict(wm_encoder_load)
else:
    wm_encoder = wm_encoder_load

wm_decoder = StegaStampDecoder(
    IMAGE_SIZE,
    IMAGE_CHANNELS,
    NUM_BITS,
)
wm_decoder_load = torch.load('models/wm_stegastamp_decoder.pth', map_location=device, weights_only=True)
if type(wm_decoder_load) is collections.OrderedDict:
    wm_decoder.load_state_dict(wm_decoder_load)
else:
    wm_encoder = wm_encoder_load

wm_encoder.to(device)
wm_decoder.to(device)


StegaStampDecoder(
  (decoder): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): ReLU()
    (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (13): ReLU()
  )
  (dense): Sequential(
    (0): Linear(in_features=8192, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=48, bias=True)
  )
)

In [8]:
class CocoCaptionMixedWMDataset(Dataset):
    def __init__(self, signature, coco_dataset, num_images):
        super(CocoCaptionMixedWMDataset, self).__init__()
        self.coco_dataset = coco_dataset
        self.dataset = []
        for i in trange(num_images):
            try:
                image, caption = coco_dataset[i]
                image = image.to(device).float()
                wm_image = wm_encoder(signature.unsqueeze(0), image.unsqueeze(0))
                self.dataset.append((wm_image.squeeze(0), 1))
                self.dataset.append((image, 0))
            except Exception as e:
                print(e)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.dataset[idx]

In [9]:
dataset = CocoCaptionMixedWMDataset(signature, coco_dataset, DATASET_SIZE)

 17%|█▋        | 172/1000 [00:06<01:09, 11.86it/s]

CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 11.06 MiB is free. Process 13172 has 14.73 GiB memory in use. Of the allocated memory 14.42 GiB is allocated by PyTorch, and 193.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 11.06 MiB is free. Process 13172 has 14.73 GiB memory in use. Of the allocated memory 14.42 GiB is allocated by PyTorch, and 193.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes

 18%|█▊        | 180/1000 [00:07<00:38, 21.23it/s]

CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 11.06 MiB is free. Process 13172 has 14.73 GiB memory in use. Of the allocated memory 14.42 GiB is allocated by PyTorch, and 193.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 11.06 MiB is free. Process 13172 has 14.73 GiB memory in use. Of the allocated memory 14.42 GiB is allocated by PyTorch, and 193.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes

 19%|█▉        | 190/1000 [00:07<00:25, 32.19it/s]

...

CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 11.06 MiB is free. Process 13172 has 14.73 GiB memory in use. Of the allocated memory 14.42 GiB is allocated by PyTorch, and 193.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 11.06 MiB is free. Process 13172 has 14.73 GiB memory in use. Of the allocated memory 14.42 GiB is allocated by PyTorch, and 193.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes

 98%|█████████▊| 982/1000 [00:22<00:00, 80.95it/s]

CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 11.06 MiB is free. Process 13172 has 14.73 GiB memory in use. Of the allocated memory 14.42 GiB is allocated by PyTorch, and 193.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 11.06 MiB is free. Process 13172 has 14.73 GiB memory in use. Of the allocated memory 14.42 GiB is allocated by PyTorch, and 193.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes

100%|██████████| 1000/1000 [00:22<00:00, 44.09it/s]

CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 11.06 MiB is free. Process 13172 has 14.73 GiB memory in use. Of the allocated memory 14.42 GiB is allocated by PyTorch, and 193.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 11.06 MiB is free. Process 13172 has 14.73 GiB memory in use. Of the allocated memory 14.42 GiB is allocated by PyTorch, and 193.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes

In [11]:
len(dataset)

336

In [12]:
import torch
from torchvision import models

resnet18 = models.resnet18(pretrained=True)
resnet18 = resnet18.to(device)
resnet18.eval()
feature_extractor = torch.nn.Sequential(*list(resnet18.children())[:-1])

print("Loaded ResNet18 model for feature extraction.")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 198MB/s]

Loaded ResNet18 model for feature extraction.


In [21]:
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=0)

features = []
labels = []
captions = []

with torch.no_grad():
    for images, lbls, caps in tqdm(dataloader, desc="Extracting Features and Captions"):
        images = images.to(device)
        outputs = feature_extractor(images)
        outputs = outputs.view(outputs.size(0), -1)  # Flatten to [batch_size, 512]
        features.append(outputs.cpu().numpy())
        labels.extend(lbls.numpy())
        captions.extend(caps)

features = np.concatenate(features, axis=0)
labels = np.array(labels)

print(f"Extracted features shape: {features.shape}")  # Expected: [2 * DATASET_SIZE, 512]
print(f"Labels shape: {labels.shape}")

  0%|          | 0/11 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 24.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 17.06 MiB is free. Process 13172 has 14.73 GiB memory in use. Of the allocated memory 14.41 GiB is allocated by PyTorch, and 195.25 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [15]:
split_idx = int(len(features) * 0.8)
X_train, X_test = features[:split_idx], features[split_idx:]
y_train, y_test = labels[:split_idx], labels[split_idx:]

clf = GaussianNB()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Precision: 1.0
Recall: 0.8235294117647058
F1 Score: 0.9032258064516129
